In [1]:
import math
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw_total_fight_data.csv', sep=';')
fighter_details = pd.read_csv('../data/raw_fighter_details.csv', index_col='fighter_name')

In [2]:
df.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,John Castaneda,Daniel Santos,1,1,57 of 106,69 of 134,53%,51%,67 of 118,71 of 137,...,2 of 2,KO/TKO,2,4:28,3 Rnd (5-5-5),Keith Peterson,"October 01, 2022","Las Vegas, Nevada, USA",Catch Weight Bout,Daniel Santos
1,Sodiq Yusuff,Don Shainis,0,0,9 of 9,3 of 3,100%,100%,9 of 9,9 of 9,...,0 of 0,Submission,1,0:30,3 Rnd (5-5-5),Jacob Montalvo,"October 01, 2022","Las Vegas, Nevada, USA",Featherweight Bout,Sodiq Yusuff
2,Raoni Barcelos,Trevin Jones,1,0,73 of 119,11 of 57,61%,19%,119 of 171,15 of 61,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"October 01, 2022","Las Vegas, Nevada, USA",Bantamweight Bout,Raoni Barcelos
3,Ilir Latifi,Aleksei Oleinik,0,0,38 of 48,28 of 70,79%,40%,74 of 88,68 of 113,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Jacob Montalvo,"October 01, 2022","Las Vegas, Nevada, USA",Heavyweight Bout,Ilir Latifi
4,Mackenzie Dern,Yan Xiaonan,0,0,61 of 151,113 of 195,40%,57%,224 of 339,151 of 240,...,7 of 8,Decision - Majority,5,5:00,5 Rnd (5-5-5-5-5),Chris Tognoni,"October 01, 2022","Las Vegas, Nevada, USA",Women's Strawweight Bout,Yan Xiaonan


In [3]:
fighter_details.drop(
    columns=["SLpM",
            "Str_Acc",
            "SApM",
            "Str_Def",
            "TD_Avg",
            "TD_Acc",
            "TD_Def",
            "Sub_Avg",
        ], inplace=True)

In [4]:
columns = ['R_SIG_STR.', 'B_SIG_STR.', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_HEAD', 'B_HEAD', 'R_BODY','B_BODY', 'R_LEG', 'B_LEG', 
        'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH','B_CLINCH', 'R_GROUND', 'B_GROUND']

In [5]:
attempt_suffix = '_att'
landed_suffix = '_landed'

for column in columns:
    df[column+attempt_suffix] = df[column].apply(lambda X: int(X.split('of')[1]))
    df[column+landed_suffix] = df[column].apply(lambda X: int(X.split('of')[0]))
    
df.drop(columns, axis=1, inplace=True)

In [6]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed'],
      dtype

In [7]:
for column in df.columns:
    if df[column].isnull().sum() != 0:
        print(f"NaN values in {column} = {df[column].isnull().sum()}")

NaN values in Referee = 32
NaN values in Winner = 121


In [8]:
df[df['Winner'].isnull()]['win_by'].value_counts()

Overturned              49
Decision - Majority     28
Could Not Continue      20
Decision - Split        15
Decision - Unanimous     6
Other                    3
Name: win_by, dtype: int64

In [9]:
df['Winner'].fillna('Draw', inplace=True)

In [10]:
pct_columns = ['R_SIG_STR_pct','B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct']

def pct_to_frac(X):
    if X != '---':
        return float(X.replace('%', ''))/100
    else:
        # if '---' means it's taking pct of `0 of 0`. 
        # Taking a call here to consider 0 landed of 0 attempted as 0 percentage
        return 0

for column in pct_columns:
    df[column] = df[column].apply(pct_to_frac)

In [11]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6797,6798,6799,6800,6801,6802,6803,6804,6805,6806
R_fighter,John Castaneda,Sodiq Yusuff,Raoni Barcelos,Ilir Latifi,Mackenzie Dern,Randy Brown,Joaquim Silva,Mike Davis,Krzysztof Jotko,Julija Stoliarenko,...,Patrick Smith,Remco Pardoel,Remco Pardoel,Johnny Rhodes,Jason DeLucia,Orlando Wiet,Frank Hamaker,Scott Morris,Patrick Smith,Johnny Rhodes
B_fighter,Daniel Santos,Don Shainis,Trevin Jones,Aleksei Oleinik,Yan Xiaonan,Francisco Trinaldo,Jesse Ronson,Viacheslav Borshchev,Brendan Allen,Chelsea Chandler,...,Scott Morris,Orlando Wiet,Alberta Cerra Leon,Fred Ettish,Scott Baker,Robert Lucarelli,Thaddeus Luster,Sean Daugherty,Ray Wizard,David Levicki
R_KD,1,0,1,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
B_KD,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.53,1.0,0.61,0.79,0.4,0.42,0.5,0.48,0.5,0.44,...,0.76,1.0,0.66,0.44,0.6,0.66,0.66,1.0,1.0,0.64
B_SIG_STR_pct,0.51,1.0,0.19,0.4,0.57,0.59,0.46,0.53,0.5,0.54,...,0.0,0.5,0.33,0.57,0.0,0.33,0.0,0.0,1.0,0.8
R_TD_pct,0.5,0.0,0.4,0.5,0.18,0.0,0.0,0.81,0.0,0.33,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
B_TD_pct,0.5,0.0,0.0,0.0,0.0,0.25,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
R_SUB_ATT,0,1,0,0,2,1,0,1,0,0,...,0,0,1,1,5,0,3,1,1,0
B_SUB_ATT,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [12]:
df['title_bout'] = df['Fight_type'].apply(lambda X: True if 'Title Bout' in X else False)

In [13]:
def make_weight_class(X):
    for weight_class in weight_classes:
        if weight_class in X:
            return weight_class
    if X == 'Catch Weight Bout' or 'Catchweight Bout':
        return 'Catch Weight'
    else:
        return 'Open Weight'

In [14]:
weight_classes = ['Women\'s Strawweight', 'Women\'s Bantamweight', 
                  'Women\'s Featherweight', 'Women\'s Flyweight', 'Lightweight', 
                  'Welterweight', 'Middleweight','Light Heavyweight', 
                  'Heavyweight', 'Featherweight','Bantamweight', 'Flyweight', 'Open Weight']

df['weight_class'] = df['Fight_type'].apply(make_weight_class)

In [15]:
df[df['weight_class'].isnull()]['Fight_type'].value_counts()

Series([], Name: Fight_type, dtype: int64)

In [16]:
df['Fight_type'].value_counts()

Lightweight Bout                                                       1150
Welterweight Bout                                                      1121
Middleweight Bout                                                       865
Featherweight Bout                                                      616
Heavyweight Bout                                                        589
                                                                       ... 
Ultimate Fighter China Welterweight Tournament Title Bout                 1
TUF Nations Canada vs. Australia Welterweight Tournament Title Bout       1
Ultimate Fighter Brazil 3 Middleweight Tournament Title Bout              1
Ultimate Fighter Brazil 3 Heavyweight Tournament Title Bout               1
UFC 2 Tournament Title Bout                                               1
Name: Fight_type, Length: 109, dtype: int64

In [17]:
df.drop(columns=['Fight_type'], inplace=True)

In [18]:
df['Format'].value_counts()

3 Rnd (5-5-5)           5997
5 Rnd (5-5-5-5-5)        590
1 Rnd + OT (12-3)         80
No Time Limit             29
3 Rnd + OT (5-5-5-5)      26
1 Rnd (20)                21
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                 8
1 Rnd (10)                 6
1 Rnd (12)                 4
1 Rnd + OT (30-5)          3
1 Rnd + OT (15-3)          2
1 Rnd (18)                 2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd (30)                 1
1 Rnd + OT (31-5)          1
1 Rnd + 2OT (24-3-3)       1
Name: Format, dtype: int64

In [19]:
time_in_first_round = {'3 Rnd (5-5-5)': 5*60, '5 Rnd (5-5-5-5-5)': 5*60, '1 Rnd + OT (12-3)': 12*60,
       'No Time Limit': 1, '3 Rnd + OT (5-5-5-5)': 5*60, '1 Rnd (20)': 1*20,
       '2 Rnd (5-5)': 5*60, '1 Rnd (15)': 15*60, '1 Rnd (10)': 10*60,
       '1 Rnd (12)':12*60, '1 Rnd + OT (30-5)': 30*60, '1 Rnd (18)': 18*60, '1 Rnd + OT (15-3)': 15*60,
       '1 Rnd (30)': 30*60, '1 Rnd + OT (31-5)': 31*5,
       '1 Rnd + OT (27-3)': 27*60, '1 Rnd + OT (30-3)': 30*60}

exception_format_time = {'1 Rnd + 2OT (15-3-3)': [15*60, 3*60], '1 Rnd + 2OT (24-3-3)': [24*60, 3*60]}

In [20]:
# Converting to seconds
df['last_round_time'] = df['last_round_time'].apply(lambda X: int(X.split(':')[0])*60 + int(X.split(':')[1]))

In [21]:
def get_total_time(row):
    if row['Format'] in time_in_first_round.keys():
        return (row['last_round'] - 1) * time_in_first_round[row['Format']] + row['last_round_time']
    elif row['Format'] in exception_format_time.keys():
        if (row['last_round'] - 1) >= 2:
            return exception_format_time[row['Format']][0] + (row['last_round'] - 2) * \
                    exception_format_time[row['Format']][1] + row['last_round_time']
        else:
            return (row['last_round'] - 1) * exception_format_time[row['Format']][0] + row['last_round_time']
    
# So if the fight ended in round 1, we only need last_round_time. 
# If it ended in round 2, we need the full time of round 1 and the last_round_time
# This works for fights with same time in each round and fights with only two rounds.

In [22]:
df['total_time_fought(seconds)'] = df.apply(get_total_time, axis=1)

In [23]:
def get_no_of_rounds(X):
    if X == 'No Time Limit':
        return 1
    else:
        return len(X.split('(')[1].replace(')', '').split('-'))

df['no_of_rounds'] = df['Format'].apply(get_no_of_rounds)

In [24]:
df.drop(columns=['Format', 'last_round_time'], inplace=True)

In [25]:
CTRL_columns = ['R_CTRL','B_CTRL']

def conv_to_sec(X):
    if X != '--':
        return int(X.split(':')[0])*60 + int(X.split(':')[1])
    else:
        # if '--' means there was no time spent on the ground. 
        # Taking a call here to consider this as 0 seconds
        return 0

for column in CTRL_columns:
    df[column+'_time(seconds)'] = df[column].apply(conv_to_sec)

In [26]:
df.drop(columns=['R_CTRL', 'B_CTRL'], inplace=True)

In [27]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'win_by', 'last_round', 'Referee', 'date', 'location',
       'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed', 'B_SIG_STR._att',
       'B_SIG_STR._landed', 'R_TOTAL_STR._att', 'R_TOTAL_STR._landed',
       'B_TOTAL_STR._att', 'B_TOTAL_STR._landed', 'R_TD_att', 'R_TD_landed',
       'B_TD_att', 'B_TD_landed', 'R_HEAD_att', 'R_HEAD_landed', 'B_HEAD_att',
       'B_HEAD_landed', 'R_BODY_att', 'R_BODY_landed', 'B_BODY_att',
       'B_BODY_landed', 'R_LEG_att', 'R_LEG_landed', 'B_LEG_att',
       'B_LEG_landed', 'R_DISTANCE_att', 'R_DISTANCE_landed', 'B_DISTANCE_att',
       'B_DISTANCE_landed', 'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att',
       'B_CLINCH_landed', 'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att',
       'B_GROUND_landed', 'title_bout', 'weight_class',
       'total_time_fought(seconds)', 'no_of_rounds',

In [27]:
df2 = df.copy()

In [29]:
df2.drop(['R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_CTRL_time(seconds)', 'B_CTRL_time(seconds)', 'R_REV', 'B_REV', 'win_by', 'last_round', 
        'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed',
        'total_time_fought(seconds)'], axis=1, inplace=True)

,0,1,2,3,4,5,6,7,8,9,...,6797,6798,6799,6800,6801,6802,6803,6804,6805,6806
R_fighter,John Castaneda,Sodiq Yusuff,Raoni Barcelos,Ilir Latifi,Mackenzie Dern,Randy Brown,Joaquim Silva,Mike Davis,Krzysztof Jotko,Julija Stoliarenko,...,Patrick Smith,Remco Pardoel,Remco Pardoel,Johnny Rhodes,Jason DeLucia,Orlando Wiet,Frank Hamaker,Scott Morris,Patrick Smith,Johnny Rhodes
B_fighter,Daniel Santos,Don Shainis,Trevin Jones,Aleksei Oleinik,Yan Xiaonan,Francisco Trinaldo,Jesse Ronson,Viacheslav Borshchev,Brendan Allen,Chelsea Chandler,...,Scott Morris,Orlando Wiet,Alberta Cerra Leon,Fred Ettish,Scott Baker,Robert Lucarelli,Thaddeus Luster,Sean Daugherty,Ray Wizard,David Levicki
Referee,Keith Peterson,Jacob Montalvo,Keith Peterson,Jacob Montalvo,Chris Tognoni,Keith Peterson,Keith Peterson,Kerry Hatley,Kerry Hatley,Jacob Montalvo,...,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy
date,"October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022",...,"March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994"
location,"Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA",...,"Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA"
Winner,Daniel Santos,Sodiq Yusuff,Raoni Barcelos,Ilir Latifi,Yan Xiaonan,Randy Brown,Joaquim Silva,Mike Davis,Brendan Allen,Chelsea Chandler,...,Patrick Smith,Remco Pardoel,Remco Pardoel,Johnny Rhodes,Jason DeLucia,Orlando Wiet,Frank Hamaker,Scott Morris,Patrick Smith,Johnny Rhodes
title_bout,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
weight_class,Catch Weight,Featherweight,Bantamweight,Heavyweight,Women's Strawweight,Welterweight,Lightweight,Lightweight,Middleweight,Catch Weight,...,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight
no_of_rounds,3,3,3,3,5,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1


In [28]:
red_fighters = df['R_fighter'].value_counts().index
blue_fighters = df['B_fighter'].value_counts().index

fighters = list(set(red_fighters) | set(blue_fighters))

In [29]:
def get_renamed_winner(row):
    if row['R_fighter'] == row['Winner']:
        return 'Red'
    elif row['B_fighter'] == row['Winner']:
        return 'Blue'
    elif row['Winner'] == 'Draw':
        return 'Draw'

df2['Winner'] = df2[['R_fighter', 'B_fighter', 'Winner']].apply(get_renamed_winner, axis=1)

In [33]:
df2.to_csv('../data/fight_arrangements.csv', index=False)

,0,1,2,3,4,5,6,7,8,9,...,6797,6798,6799,6800,6801,6802,6803,6804,6805,6806
R_fighter,John Castaneda,Sodiq Yusuff,Raoni Barcelos,Ilir Latifi,Mackenzie Dern,Randy Brown,Joaquim Silva,Mike Davis,Krzysztof Jotko,Julija Stoliarenko,...,Patrick Smith,Remco Pardoel,Remco Pardoel,Johnny Rhodes,Jason DeLucia,Orlando Wiet,Frank Hamaker,Scott Morris,Patrick Smith,Johnny Rhodes
B_fighter,Daniel Santos,Don Shainis,Trevin Jones,Aleksei Oleinik,Yan Xiaonan,Francisco Trinaldo,Jesse Ronson,Viacheslav Borshchev,Brendan Allen,Chelsea Chandler,...,Scott Morris,Orlando Wiet,Alberta Cerra Leon,Fred Ettish,Scott Baker,Robert Lucarelli,Thaddeus Luster,Sean Daugherty,Ray Wizard,David Levicki
Referee,Keith Peterson,Jacob Montalvo,Keith Peterson,Jacob Montalvo,Chris Tognoni,Keith Peterson,Keith Peterson,Kerry Hatley,Kerry Hatley,Jacob Montalvo,...,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy,John McCarthy
date,"October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022","October 01, 2022",...,"March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994","March 11, 1994"
location,"Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA","Las Vegas, Nevada, USA",...,"Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA","Denver, Colorado, USA"
Winner,Blue,Red,Red,Red,Blue,Red,Red,Red,Blue,Blue,...,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red
title_bout,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
weight_class,Catch Weight,Featherweight,Bantamweight,Heavyweight,Women's Strawweight,Welterweight,Lightweight,Lightweight,Middleweight,Catch Weight,...,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight
no_of_rounds,3,3,3,3,5,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1


In [30]:
df = pd.concat([df,pd.get_dummies(df['win_by'], prefix='win_by')],axis=1)
df.drop(['win_by'],axis=1, inplace=True)

In [31]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'last_round', 'Referee', 'date', 'location', 'Winner',
       'R_SIG_STR._att', 'R_SIG_STR._landed', 'B_SIG_STR._att',
       'B_SIG_STR._landed', 'R_TOTAL_STR._att', 'R_TOTAL_STR._landed',
       'B_TOTAL_STR._att', 'B_TOTAL_STR._landed', 'R_TD_att', 'R_TD_landed',
       'B_TD_att', 'B_TD_landed', 'R_HEAD_att', 'R_HEAD_landed', 'B_HEAD_att',
       'B_HEAD_landed', 'R_BODY_att', 'R_BODY_landed', 'B_BODY_att',
       'B_BODY_landed', 'R_LEG_att', 'R_LEG_landed', 'B_LEG_att',
       'B_LEG_landed', 'R_DISTANCE_att', 'R_DISTANCE_landed', 'B_DISTANCE_att',
       'B_DISTANCE_landed', 'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att',
       'B_CLINCH_landed', 'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att',
       'B_GROUND_landed', 'title_bout', 'weight_class',
       'total_time_fought(seconds)', 'no_of_rounds', 'R_CTRL_t

In [32]:
Numerical_columns = ['hero_KD', 'opp_KD', 'hero_SIG_STR_pct',
       'opp_SIG_STR_pct', 'hero_TD_pct', 'opp_TD_pct', 'hero_SUB_ATT', 'opp_SUB_ATT',
        'hero_REV', 'opp_REV', 'hero_SIG_STR._att', 'hero_SIG_STR._landed',
       'opp_SIG_STR._att', 'opp_SIG_STR._landed', 'hero_TOTAL_STR._att',
       'hero_TOTAL_STR._landed', 'opp_TOTAL_STR._att', 'opp_TOTAL_STR._landed',
       'hero_TD_att', 'hero_TD_landed', 'opp_TD_att', 'opp_TD_landed', 'hero_HEAD_att',
       'hero_HEAD_landed', 'opp_HEAD_att', 'opp_HEAD_landed', 'hero_BODY_att',
       'hero_BODY_landed', 'opp_BODY_att', 'opp_BODY_landed', 'hero_LEG_att',
       'hero_LEG_landed', 'opp_LEG_att', 'opp_LEG_landed', 'hero_DISTANCE_att',
       'hero_DISTANCE_landed', 'opp_DISTANCE_att', 'opp_DISTANCE_landed',
       'hero_CLINCH_att', 'hero_CLINCH_landed', 'opp_CLINCH_att', 'opp_CLINCH_landed',
       'hero_GROUND_att', 'hero_GROUND_landed', 'opp_GROUND_att', 'opp_GROUND_landed', 
        'hero_CTRL_time(seconds)', 'opp_CTRL_time(seconds)',
       'total_time_fought(seconds)']

Categorical_columns = ['win_by', 'last_round',
        'Winner', 'title_bout']

In [33]:
import re

def lreplace(pattern, sub, string):
    """
    Replaces 'pattern' in 'string' with 'sub' if 'pattern' starts 'string'.
    """
    return re.sub('^%s' % pattern, sub, string)

In [34]:
red = df.groupby('R_fighter')
blue = df.groupby('B_fighter')

In [35]:
def get_fighter_red(fighter_name):
    try:
        fighter_red = red.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_red.columns:
        if re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'hero_', column)
        elif re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'opp_', column)
    fighter_red = fighter_red.rename(rename_columns, axis='columns')
    return fighter_red

In [36]:
def get_fighter_blue(fighter_name):
    try:
        fighter_blue = blue.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_blue.columns:
        if re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'hero_', column)
        elif re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'opp_', column)
    fighter_blue = fighter_blue.rename(rename_columns, axis='columns')
    return fighter_blue

In [37]:
def get_result_stats(result_list):
    result_list.reverse() # To get it in ascending order
    current_win_streak = 0
    current_lose_streak = 0
    longest_win_streak = 0
    wins = 0
    losses = 0
    draw = 0
    for result in result_list:
        if result == 'hero':
            wins += 1
            current_win_streak += 1
            current_lose_streak = 0
            if longest_win_streak < current_win_streak:
                longest_win_streak += 1
        elif result == 'opp':
            losses += 1
            current_win_streak = 0
            current_lose_streak += 1
        elif result == 'draw':
            draw += 1
            current_lose_streak = 0
            current_win_streak = 0
            
    return current_win_streak, current_lose_streak, longest_win_streak, wins, losses, draw

In [38]:
win_by_columns = ['win_by_Decision - Majority', 'win_by_Decision - Split',
       'win_by_Decision - Unanimous', 'win_by_KO/TKO','win_by_Submission',
       'win_by_TKO - Doctor\'s Stoppage']

In [39]:
temp_blue_frame = pd.DataFrame()
temp_red_frame = pd.DataFrame()
result_stats = ['current_win_streak', 'current_lose_streak', 'longest_win_streak', 'wins', 'losses', 'draw']

for fighter_name in fighters:
    fighter_red = get_fighter_red(fighter_name)
    fighter_blue = get_fighter_blue(fighter_name)
    fighter_index = None
    
    if fighter_red is None:
        fighter = fighter_blue
        fighter_index = 'blue'
    elif fighter_blue is None:
        fighter = fighter_red
        fighter_index = 'red'
    else:
        fighter = pd.concat([fighter_red, fighter_blue]).sort_index()
    
    fighter['Winner'] = fighter['Winner'].apply(lambda X: 'hero' if X == fighter_name else 'opp')

    for i, index in enumerate(fighter.index):
        fighter_slice = fighter[(i+1):].sort_index(ascending=False)
        s = fighter_slice[Numerical_columns].ewm(span=3, adjust=False).mean().tail(1)
        if len(s) != 0:
            pass
        else:
            s.loc[len(s)] = [np.NaN for _ in s.columns]
        s['total_rounds_fought'] = fighter_slice['last_round'].sum()
        s['total_title_bouts'] = fighter_slice[fighter_slice['title_bout']==True]['title_bout'].count()
        s['hero_fighter'] = fighter_name
        results = get_result_stats(list(fighter_slice['Winner']))
        for result_stat, result in zip(result_stats, results):
            s[result_stat] = result
        win_by_results = fighter_slice[fighter_slice['Winner'] == 'hero'][win_by_columns].sum()
        for win_by_column,win_by_result in zip(win_by_columns, win_by_results):
            s[win_by_column] = win_by_result
        s.index = [index]


        if fighter_index is None:
            if index in fighter_blue.index:
                temp_blue_frame = pd.concat([temp_blue_frame, s])
            elif index in fighter_red.index:
                temp_red_frame = pd.concat([temp_red_frame, s])
        elif fighter_index == 'blue':
            temp_blue_frame = pd.concat([temp_blue_frame, s])
        elif fighter_index == 'red':
            temp_red_frame = pd.concat([temp_red_frame, s])

In [42]:
len(fighters)

2295

In [45]:
fighter_details = fighter_details[fighter_details.index.isin(fighters)]
for col in fighter_details.columns:
    print(f"Number of NaN in {col} : {fighter_details[col].isnull().sum()}")

Number of NaN in Height : 13
Number of NaN in Weight : 10
Number of NaN in Reach : 641
Number of NaN in Stance : 75
Number of NaN in DOB : 141


In [46]:
fighter_details.columns

Index(['Height', 'Weight', 'Reach', 'Stance', 'DOB'], dtype='object')

In [47]:
def convert_to_cms(X):
    if X is np.NaN:
        return X
    elif len(X.split("'")) == 2:
        feet = float(X.split("'")[0])
        inches = int(X.split("'")[1].replace(' ', '').replace('"',''))
        return (feet * 30.48) + (inches * 2.54)
    else:
        return float(X.replace('"','')) * 2.54

In [48]:
fighter_details['Height_cms'] = fighter_details['Height'].apply(convert_to_cms)
fighter_details['Reach_cms'] = fighter_details['Reach'].apply(convert_to_cms)

In [49]:
fighter_details['Weight_lbs'] = fighter_details['Weight'].apply(lambda X: float(X.replace(' lbs.', '')) if X is not np.NaN else X)

In [50]:
fighter_details.drop(['Height', 'Weight', 'Reach'], axis=1, inplace=True)

In [51]:
fighter_details.reset_index(inplace=True)
temp_red_frame.reset_index(inplace=True)
temp_blue_frame.reset_index(inplace=True)

In [52]:
temp_blue_frame = temp_blue_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_blue_frame.set_index('index', inplace=True)

In [53]:
temp_blue_frame[['hero_fighter', 'fighter_name', 'Height_cms', 'Weight_lbs', 'DOB']].head(20)

,hero_fighter,fighter_name,Height_cms,Weight_lbs,DOB
index,,,,,
3858,Tim Gorman,Tim Gorman,165.10,135.0,"Apr 29, 1983"
4162,Tim Gorman,Tim Gorman,165.10,135.0,"Apr 29, 1983"
5291,Tim Hague,Tim Hague,193.04,260.0,"May 09, 1983"
5478,Tim Hague,Tim Hague,193.04,260.0,"May 09, 1983"
5538,Tim Hague,Tim Hague,193.04,260.0,"May 09, 1983"
5645,Tim Hague,Tim Hague,193.04,260.0,"May 09, 1983"
155,Sergey Morozov,Sergey Morozov,167.64,135.0,"Jun 14, 1989"
364,Sergey Morozov,Sergey Morozov,167.64,135.0,"Jun 14, 1989"
633,Sergey Morozov,Sergey Morozov,167.64,135.0,"Jun 14, 1989"


In [54]:
temp_red_frame = temp_red_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_red_frame.set_index('index', inplace=True)

In [55]:
temp_blue_frame.drop('fighter_name', axis=1, inplace=True)
temp_red_frame.drop('fighter_name', axis=1, inplace=True)

In [56]:
blue_frame = temp_blue_frame.add_prefix('B_')
red_frame = temp_red_frame.add_prefix('R_')

In [57]:
frame = blue_frame.join(red_frame, how='outer')

In [58]:
rename_cols = {}
for col in frame.columns:
    if 'hero' in col:
        rename_cols[col] = col.replace('_hero_', '_avg_').replace('.', '')
    if 'opp' in col:
        rename_cols[col] = col.replace('_opp_', '_avg_opp_').replace('.', '')
    if 'win_by' in col:
        rename_cols[col] = col.replace(' ', '').replace('-', '_').replace('\'s', '_')

In [59]:
frame.rename(rename_cols, axis='columns', inplace=True)

In [60]:
frame.drop(['R_avg_fighter','B_avg_fighter'], axis=1, inplace=True)

In [ ]:
df2.columns

In [61]:
df2 = df2.join(frame, how='outer')

In [62]:
df2['R_DOB'] = pd.to_datetime(df2['R_DOB'])
df2['B_DOB'] = pd.to_datetime(df2['B_DOB'])
df2['date'] = pd.to_datetime(df2['date'])

In [63]:
def get_age(row):
    B_age = (row['date'] - row['B_DOB']).days
    R_age = (row['date'] - row['R_DOB']).days
    if np.isnan(B_age)!=True:
        B_age = math.floor(B_age/365.25)
    if np.isnan(R_age)!=True:
        R_age = math.floor(R_age/365.25)
    return pd.Series([B_age, R_age], index=['B_age', 'R_age'])

In [64]:
df2[['B_age', 'R_age']]= df2[['date', 'R_DOB', 'B_DOB']].apply(get_age, axis=1)
df2.drop(['R_DOB', 'B_DOB'], axis=1, inplace=True)
df2.to_csv('../data/data.csv', index=False)

In [65]:
df2.columns

Index(['R_fighter', 'B_fighter', 'Referee', 'date', 'location', 'Winner',
       'title_bout', 'weight_class', 'no_of_rounds', 'B_avg_KD',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=145)